# Topic 8: SEO & Link Analysis

## Objectives:
- Implement PageRank algorithm
- Implement HITS (Hyperlink-Induced Topic Search)
- Link analysis for Vietnamese football articles
- Visualize authority and hub scores

**Dataset**: 1,830 Vietnamese football articles from VnExpress

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from tqdm import tqdm
import re
from pathlib import Path

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")

## 2. Load Vietnamese Football Articles

In [ ]:
# Load all data files
data_dir = Path('../data/raw')
all_documents = []

for i in range(1, 5):
    file_path = data_dir / f'vnexpressT_bongda_part{i}.json'
    with open(file_path, 'r', encoding='utf-8') as f:
        docs = json.load(f)
        all_documents.extend(docs)

print(f"✓ Loaded {len(all_documents)} articles")
print(f"Sample document keys: {list(all_documents[0].keys())}")

# Display first article
print("\n📰 Sample Article:")
print(f"Title: {all_documents[0].get('title', 'N/A')[:100]}...")
print(f"URL: {all_documents[0].get('url', 'N/A')}")

## 3. Extract Links & Build Link Graph

Create a graph where:
- Nodes = Articles
- Edges = Links between articles (based on content similarity or actual hyperlinks)

In [ ]:
# Extract entities (teams, players) to create semantic links
def extract_entities(text):
    """Extract Vietnamese football entities"""
    if not text:
        return set()
    
    text = text.lower()
    entities = set()
    
    # Teams
    teams = ['manchester united', 'barcelona', 'real madrid', 'liverpool', 
             'chelsea', 'arsenal', 'bayern', 'psg', 'juventus',
             'việt nam', 'hà nội', 'tp.hcm', 'sài gòn', 'viettel']
    
    # Players
    players = ['ronaldo', 'messi', 'neymar', 'mbappé', 'haaland',
               'quang hải', 'công phượng', 'văn quyết', 'xuân trường']
    
    # Competitions
    competitions = ['world cup', 'champions league', 'premier league', 
                   'la liga', 'v-league', 'sea games', 'aff cup']
    
    all_entities = teams + players + competitions
    
    for entity in all_entities:
        if entity in text:
            entities.add(entity)
    
    return entities

# Build link graph
print("Building link graph...")
G = nx.DiGraph()

# Add all documents as nodes
for i, doc in enumerate(tqdm(all_documents, desc="Adding nodes")):
    title = doc.get('title', f'Doc_{i}')
    content = doc.get('content', '') or doc.get('text', '')
    entities = extract_entities(title + ' ' + content)
    
    G.add_node(i, 
               title=title[:50],  # Truncate for display
               entities=entities,
               url=doc.get('url', ''))

# Create edges based on shared entities
print("\nCreating edges based on shared entities...")
edge_count = 0
for i in tqdm(range(len(all_documents)), desc="Building edges"):
    entities_i = G.nodes[i]['entities']
    if not entities_i:
        continue
    
    for j in range(i + 1, min(i + 50, len(all_documents))):  # Limit for performance
        entities_j = G.nodes[j]['entities']
        
        # If they share 2+ entities, create a link
        shared = entities_i & entities_j
        if len(shared) >= 2:
            G.add_edge(i, j, weight=len(shared))
            G.add_edge(j, i, weight=len(shared))  # Bidirectional
            edge_count += 2

print(f"\n✓ Graph created:")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")
print(f"  Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")

## 4. PageRank Algorithm

**Formula**: 
$$PR(A) = (1-d) + d \sum_{i=1}^n \frac{PR(T_i)}{C(T_i)}$$

Where:
- $d$ = damping factor (0.85)
- $T_i$ = pages linking to page A
- $C(T_i)$ = number of outbound links from $T_i$

In [ ]:
def pagerank(G, damping=0.85, max_iter=100, tol=1e-6):
    """
    Implement PageRank algorithm from scratch
    """
    N = G.number_of_nodes()
    if N == 0:
        return {}
    
    # Initialize PageRank scores
    pr = {node: 1.0 / N for node in G.nodes()}
    
    for iteration in range(max_iter):
        pr_new = {}
        
        for node in G.nodes():
            # Sum of PR from incoming links
            incoming_sum = 0
            for predecessor in G.predecessors(node):
                out_degree = G.out_degree(predecessor)
                if out_degree > 0:
                    incoming_sum += pr[predecessor] / out_degree
            
            # PageRank formula
            pr_new[node] = (1 - damping) / N + damping * incoming_sum
        
        # Check convergence
        diff = sum(abs(pr_new[node] - pr[node]) for node in G.nodes())
        pr = pr_new
        
        if diff < tol:
            print(f"✓ Converged after {iteration + 1} iterations")
            break
    
    return pr

# Calculate PageRank
print("Calculating PageRank...")
pr_scores = pagerank(G, damping=0.85)

# Compare with NetworkX implementation
pr_nx = nx.pagerank(G, alpha=0.85)

print("\n📊 Top 10 Articles by PageRank:")
top_pr = sorted(pr_scores.items(), key=lambda x: x[1], reverse=True)[:10]
for rank, (node_id, score) in enumerate(top_pr, 1):
    title = G.nodes[node_id]['title']
    print(f"{rank}. {title[:60]}... (PR={score:.6f})")

## 5. HITS Algorithm (Hubs & Authorities)

**Formulas**:
- Authority: $a(p) = \sum_{q \in B(p)} h(q)$
- Hub: $h(p) = \sum_{q \in F(p)} a(q)$

Where:
- $B(p)$ = pages pointing to $p$
- $F(p)$ = pages that $p$ points to

In [ ]:
def hits(G, max_iter=100, tol=1e-6):
    """
    Implement HITS algorithm from scratch
    """
    N = G.number_of_nodes()
    if N == 0:
        return {}, {}
    
    # Initialize hub and authority scores
    hubs = {node: 1.0 for node in G.nodes()}
    authorities = {node: 1.0 for node in G.nodes()}
    
    for iteration in range(max_iter):
        # Update authority scores
        auth_new = {}
        for node in G.nodes():
            auth_new[node] = sum(hubs[predecessor] 
                                for predecessor in G.predecessors(node))
        
        # Update hub scores
        hubs_new = {}
        for node in G.nodes():
            hubs_new[node] = sum(auth_new[successor] 
                                for successor in G.successors(node))
        
        # Normalize
        auth_norm = np.sqrt(sum(v**2 for v in auth_new.values()))
        hub_norm = np.sqrt(sum(v**2 for v in hubs_new.values()))
        
        if auth_norm > 0:
            auth_new = {k: v / auth_norm for k, v in auth_new.items()}
        if hub_norm > 0:
            hubs_new = {k: v / hub_norm for k, v in hubs_new.items()}
        
        # Check convergence
        diff_auth = sum(abs(auth_new[node] - authorities[node]) for node in G.nodes())
        diff_hub = sum(abs(hubs_new[node] - hubs[node]) for node in G.nodes())
        
        authorities = auth_new
        hubs = hubs_new
        
        if diff_auth < tol and diff_hub < tol:
            print(f"✓ HITS converged after {iteration + 1} iterations")
            break
    
    return hubs, authorities

# Calculate HITS
print("Calculating HITS...")
hub_scores, auth_scores = hits(G)

# Compare with NetworkX
hits_nx = nx.hits(G, max_iter=100)
hubs_nx, auth_nx = hits_nx

print("\n📊 Top 10 Authority Articles (Most Referenced):")
top_auth = sorted(auth_scores.items(), key=lambda x: x[1], reverse=True)[:10]
for rank, (node_id, score) in enumerate(top_auth, 1):
    title = G.nodes[node_id]['title']
    print(f"{rank}. {title[:60]}... (Auth={score:.6f})")

print("\n📊 Top 10 Hub Articles (Most Outgoing Links):")
top_hubs = sorted(hub_scores.items(), key=lambda x: x[1], reverse=True)[:10]
for rank, (node_id, score) in enumerate(top_hubs, 1):
    title = G.nodes[node_id]['title']
    print(f"{rank}. {title[:60]}... (Hub={score:.6f})")

## 6. Visualization & Comparison

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'node_id': list(G.nodes()),
    'title': [G.nodes[i]['title'] for i in G.nodes()],
    'pagerank': [pr_scores.get(i, 0) for i in G.nodes()],
    'authority': [auth_scores.get(i, 0) for i in G.nodes()],
    'hub': [hub_scores.get(i, 0) for i in G.nodes()],
    'in_degree': [G.in_degree(i) for i in G.nodes()],
    'out_degree': [G.out_degree(i) for i in G.nodes()]
})

# Sort by PageRank
comparison_df = comparison_df.sort_values('pagerank', ascending=False)

print("\n📊 Top 20 Articles - All Metrics:")
print(comparison_df.head(20).to_string(index=False))

# Save to CSV
output_dir = Path('../outputs/seo_analysis')
output_dir.mkdir(parents=True, exist_ok=True)
comparison_df.to_csv(output_dir / 'link_analysis_results.csv', index=False, encoding='utf-8')
print(f"\n✓ Results saved to {output_dir / 'link_analysis_results.csv'}")

In [ ]:
# Plot 1: Score Distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# PageRank distribution
axes[0, 0].hist(comparison_df['pagerank'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('PageRank Score Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('PageRank Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(alpha=0.3)

# Authority distribution
axes[0, 1].hist(comparison_df['authority'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Authority Score Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Authority Score')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(alpha=0.3)

# Hub distribution
axes[1, 0].hist(comparison_df['hub'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1, 0].set_title('Hub Score Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Hub Score')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(alpha=0.3)

# Degree distribution
axes[1, 1].scatter(comparison_df['in_degree'], comparison_df['out_degree'], alpha=0.5)
axes[1, 1].set_title('In-Degree vs Out-Degree', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('In-Degree')
axes[1, 1].set_ylabel('Out-Degree')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'score_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Score distributions saved")

In [ ]:
# Plot 2: Correlation between metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# PageRank vs Authority
axes[0].scatter(comparison_df['pagerank'], comparison_df['authority'], alpha=0.5)
axes[0].set_title('PageRank vs Authority', fontsize=14, fontweight='bold')
axes[0].set_xlabel('PageRank')
axes[0].set_ylabel('Authority')
axes[0].grid(alpha=0.3)
corr1 = np.corrcoef(comparison_df['pagerank'], comparison_df['authority'])[0, 1]
axes[0].text(0.05, 0.95, f'Correlation: {corr1:.3f}', 
            transform=axes[0].transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# PageRank vs Hub
axes[1].scatter(comparison_df['pagerank'], comparison_df['hub'], alpha=0.5, color='orange')
axes[1].set_title('PageRank vs Hub', fontsize=14, fontweight='bold')
axes[1].set_xlabel('PageRank')
axes[1].set_ylabel('Hub')
axes[1].grid(alpha=0.3)
corr2 = np.corrcoef(comparison_df['pagerank'], comparison_df['hub'])[0, 1]
axes[1].text(0.05, 0.95, f'Correlation: {corr2:.3f}', 
            transform=axes[1].transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Authority vs Hub
axes[2].scatter(comparison_df['authority'], comparison_df['hub'], alpha=0.5, color='green')
axes[2].set_title('Authority vs Hub', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Authority')
axes[2].set_ylabel('Hub')
axes[2].grid(alpha=0.3)
corr3 = np.corrcoef(comparison_df['authority'], comparison_df['hub'])[0, 1]
axes[2].text(0.05, 0.95, f'Correlation: {corr3:.3f}', 
            transform=axes[2].transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig(output_dir / 'metric_correlations.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Correlation plots saved")

In [ ]:
# Plot 3: Visualize small subgraph
# Get top 30 nodes by PageRank
top_30_nodes = list(comparison_df.head(30)['node_id'])
subgraph = G.subgraph(top_30_nodes)

plt.figure(figsize=(16, 12))
pos = nx.spring_layout(subgraph, k=2, iterations=50, seed=42)

# Node sizes based on PageRank
node_sizes = [pr_scores[node] * 50000 for node in subgraph.nodes()]

# Node colors based on Authority
node_colors = [auth_scores[node] for node in subgraph.nodes()]

# Draw network
nx.draw_networkx_nodes(subgraph, pos, 
                       node_size=node_sizes,
                       node_color=node_colors,
                       cmap='YlOrRd',
                       alpha=0.7)

nx.draw_networkx_edges(subgraph, pos, 
                       alpha=0.2,
                       edge_color='gray',
                       arrows=True,
                       arrowsize=10)

# Add labels for top 10 nodes
top_10_nodes = list(comparison_df.head(10)['node_id'])
labels = {node: G.nodes[node]['title'][:20] + '...' 
          for node in subgraph.nodes() if node in top_10_nodes}
nx.draw_networkx_labels(subgraph, pos, labels, font_size=8)

plt.title('Top 30 Articles Link Network\n(Size=PageRank, Color=Authority)', 
         fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.savefig(output_dir / 'link_network_visualization.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Network visualization saved")

## 7. Summary Statistics

In [ ]:
print("\n" + "="*80)
print("📊 LINK ANALYSIS SUMMARY")
print("="*80)

print("\n📈 Graph Statistics:")
print(f"  Total nodes: {G.number_of_nodes()}")
print(f"  Total edges: {G.number_of_edges()}")
print(f"  Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
print(f"  Graph density: {nx.density(G):.6f}")

print("\n📊 PageRank Statistics:")
print(f"  Mean: {comparison_df['pagerank'].mean():.6f}")
print(f"  Std:  {comparison_df['pagerank'].std():.6f}")
print(f"  Max:  {comparison_df['pagerank'].max():.6f}")
print(f"  Min:  {comparison_df['pagerank'].min():.6f}")

print("\n📊 Authority Statistics:")
print(f"  Mean: {comparison_df['authority'].mean():.6f}")
print(f"  Std:  {comparison_df['authority'].std():.6f}")
print(f"  Max:  {comparison_df['authority'].max():.6f}")
print(f"  Min:  {comparison_df['authority'].min():.6f}")

print("\n📊 Hub Statistics:")
print(f"  Mean: {comparison_df['hub'].mean():.6f}")
print(f"  Std:  {comparison_df['hub'].std():.6f}")
print(f"  Max:  {comparison_df['hub'].max():.6f}")
print(f"  Min:  {comparison_df['hub'].min():.6f}")

print("\n📊 Correlations:")
print(f"  PageRank vs Authority: {np.corrcoef(comparison_df['pagerank'], comparison_df['authority'])[0,1]:.3f}")
print(f"  PageRank vs Hub:       {np.corrcoef(comparison_df['pagerank'], comparison_df['hub'])[0,1]:.3f}")
print(f"  Authority vs Hub:      {np.corrcoef(comparison_df['authority'], comparison_df['hub'])[0,1]:.3f}")

print("\n✅ Topic 8 Complete: SEO & Link Analysis")
print("="*80)